In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
import cv2
import math

# Topic 1: Foundations


## Transformations, Projections, Meshes



### Transformation Matrices




In [ ]:
# 2D Homogeneous Transformations

def translation2D(x, y, dx, dy):
  X = np.vstack((x, y, 1))
  T = np.array([
      [1, 0, dx],
      [0, 1, dy],
      [0, 0, 1]
  ])
  return T @ X

def scaling2D(x, y, sx, sy):
  X = np.vstack((x, y, 1))
  T = np.array([
      [sx, 0, 0],
      [0, sy, 0],
      [0, 0, 1]
  ])
  return T @ X

def rotation2D(x, y, theta):
  X = np.vstack((x, y, 1))
  T = np.array([
      [np.cos(theta), -np.sin(theta), 0],
      [np.sin(theta), np.cos(theta), 0],
      [0, 0, 1]
  ])
  return T @ X

In [ ]:
# 3D Homogeneous Transformations

def translation3D(x, y, z, dx, dy, dz):
  X = np.vstack((x, y, z, 1))
  T = np.array([
      [1, 0, 0, dx],
      [0, 1, 0, dy],
      [0, 0, 1, dz],
      [0, 0, 0, 1]
  ])
  return T @ X

def scaling3D(x, y, z, sx, sy, sz):
  X = np.vstack((x, y, z, 1))
  T = np.array([
      [sx, 0, 0, 0],
      [0, sy, 0, 0],
      [0, 0, sz, 0],
      [0, 0, 0, 1]
  ])
  return T @ X

def rotation3DX(x, y, z, theta):
  X = np.vstack((x, y, z, 1))
  T = np.array([
      [1, 0, 0, 0],
      [0, np.cos(theta), -np.sin(theta), 0],
      [0, np.sin(theta), np.cos(theta), 0],
      [0, 0, 0, 1]
  ])
  return T @ X

def rotation3DY(x, y, z, theta):
  X = np.vstack((x, y, z, 1))
  T = np.array([
      [np.cos(theta), 0, np.sin(theta), 0],
      [0, 1, 0, 0],
      [-np.sin(theta), 0, np.cos(theta), 0],
      [0, 0, 0, 1]
  ])
  return T @ X

def rotation3DZ(x, y, z, theta):
  X = np.vstack((x, y, z, 1))
  T = np.array([
      [np.cos(theta), -np.sin(theta), 0, 0],
      [np.sin(theta), np.cos(theta), 0, 0],
      [0, 0, 1, 0],
      [0, 0, 0, 1]
  ])
  return T @ X

### Perspective Projection



- *x, y* : image coordinates
- *u, v, w* : object coordinates
- *U, V, W* : world coordinates
- K = ⟨ f, c<sub>x</sub>, c<sub>y</sub> ⟩ : intrinsic parameters
  - *f = sd* : focal length
    - *d* : phyiscal focal length (mm)
    - *s* : scale factor (pixels per mm)
  - *c<sub>x</sub>, c<sub>y</sub>* = principle point offsets
- *R* : camera rotation matrix
- *t* : camera translation matrix
- *λ* : coordinate scalefactor


---

\begin{align}
        x = \frac{u'}{w'} = \frac{fu}{w} + c_x
    \end{align}

\begin{align}
        y = \frac{v'}{w'} = \frac{fv}{w} + c_y
    \end{align}

\begin{align}
        \left [
        \begin{array}{cl}
            u' \\
            v' \\
            w' \\
            \end{array}
            \right ]=
        λ\left [
            \begin{array}{cl}
            u \\
            v \\
            w \\
            \end{array}
          \right ] = K \left [
          \begin{array}{cl}
          R t
          \end{array}
          \right ]
          \left [
            \begin{array}{cl}
            U \\
            V \\
            W \\
            \end{array}
          \right ]
    \end{align}

In [ ]:
def intrinsic_parameters(f, cx, cy, fx = None, fy = None):
  if not (fx and fy):
    fx = f
    fy = f
  return np.array([
      [fx, 0, cx],
      [0, fy, cx],
      [0, 0, 1]
  ])

def extrinsic_parameters(R, t):
  return np.hstack((R, t))

def pp_homogeneous_coordinates(K, R, t, U, V, W):
  K_h = np.c_[ K, np.zeros(3) ]
  R_t_h = np.r_[ np.c_[ R, t ], np.array([0, 0, 0, 1])]
  W = np.vstack((U, V, W, 1))
  lambda_X = K_h @ (R_t_h @ W)
  return lambda_X

## Radiometry, Photometry, Colour, Human Vision and Digital Cameras.



### Radiometry and Photometry



**Radiant Energy:** A photon of wavelength λ carries some energy Q, measured in Joules (J).

\begin{align}
      Q = \frac{hc}{λ}
    \end{align}

- *h* is Plank's constant.
- *c* is the speed of light.

**Radiant Flux:** The total amount of energy passing through a surface or region of space per unit time, measured in Watts (W).

\begin{align}
        Φ = ∫_0^{Δt} \frac{ΔQ}{Δt}=\frac{dQ}{dt}
    \end{align}

**Irradiance:** The power incident of a surface per unit area, measured in W/m<sup>2</sup>

Irradiance of a flat surface:
\begin{align}
      E = \frac{dΦ}{dA}
\end{align}

Irradiance of a sphere:
\begin{align}
      E = \frac{dΦ}{4πr^2}
\end{align}

Irradiance of a point light source that illuminates uniformly (isotropic) between the angle of the light direction and surface normal, θ:
\begin{align}
      E = \frac{dΦcosθ}{dA}
\end{align}

**Radiant Exitance:** The power per unit area leaving a surface, measured in W/m<sup>2</sup>. Mathematically it is the same as the Irradiance.

**Angles:**

2D: The Angle of the subtended arclength on a circle to raidus.
\begin{align}
      dθ = \frac{dl}{r}
\end{align}

3D: The Solid Angle of the subtended area on a sphere to radius squared.
\begin{align}
      dω = \frac{A}{r^2}
\end{align}

**Radiant Intensity:** The light emitted into a particular direction from an infinitesimally small point light source is called Radiant Intensity. Measured in energy per unit of time per unit of direction (W/sr).
\begin{align}
      I(ω) = \frac{dΦ}{dω}
\end{align}

**Radiance:** The flux per unit area normal to a beam. Measured in Watts per unit area per unit direction (W/m<sup>2</sup>sr).

\begin{align}
      L = \frac{d^2Φ}{dA × cosθ × dω}
\end{align}

- θ is the angle between the normal of the surface area element dA and the direction of the flux Φ.
- The cosine term represent the foreshortening with respect to the flux direction.

Radiance measures the flux passing through, leaving or arriving at a point in a particular direction.

**Spectral Power Distribution (SPD):** The radiometric quantities as a function of wavelength. Spectral radiance, L<sub>λ</sub> is the radiance per unit wavelength interval and is measured in (W/m<sup>3</sup>sr).

**Radiometry**

- Radiant energy (Q) = Joule (J)
- Radiant flux (Φ) = Watt (W)
- Intensity (I) = W/sr
- Irradiance (E) = W/m<sup>2</sup>
- Radiance (L) = W/m<sup>2</sup>sr

**Photometry**

- Luminous energy = Talbot (T)
- Luminous flux = lumen (lm)
- Luminous intensity = lm/sr = candela (cd)
- Illuminance = lm/m<sup>2</sup> = lux (lx)
- Luminance (Y) = lm/m<sup>2</sup>sr = cd/m<sup>2</sup> = nit

**Luminance (Y):** Measure of how bright a SPD appears to a human observer.

\begin{align}
      Y = ∫_{380}^{830} L_λV(λ)Δλ
\end{align}

- L<sub>λ</sub> = Spectral Distrubution
- V(λ) = Spectral luminous efficiency function

### Colour and Colour Spaces



For each spectral target Q<sub>λ</sub>, the intensity for each primary RGB colour can be matched with colour matching functions: r(λ), g(λ), b(λ) to adjust the intensity of R, G and B.

\begin{align}
      Q_λ = r(λ)R + g(λ)G + b(λ)B
\end{align}

The CIE has defined standard colour matching functions: x̄(λ), ȳ(λ), z̄(λ)

**Emissive case:** Given the SPD of a lightsource I(λ) and the CIE colour matching functions, the XYZ tristimulus values are computed as:

\begin{align}
      X = ∫_λ I(λ) × x̄(λ) × Δλ \\
      Y = ∫_λ I(λ) × ȳ(λ) × Δλ \\
      Z = ∫_λ I(λ) × z̄(λ) × Δλ
\end{align}

**Reflective and Transmissive cases:** Given the SPD of a lightsource I(λ), spectral reflectance of a material (spectral albedo) P(λ), and the CIE colour matching functions, the XYZ tristimulus values are computed as:

\begin{align}
      X = \frac{1}{N} ∫_λ I(λ) × P(λ) × x̄(λ) × Δλ \\
      Y = \frac{1}{N} ∫_λ I(λ) × P(λ) × ȳ(λ) × Δλ \\
      Z = \frac{1}{N} ∫_λ I(λ) × P(λ) × z̄(λ) × Δλ
\end{align}

Where N = ∫<sub>λ</sub> I(λ) × ȳ(λ) × Δλ, λ ∈ visible.

**CIE xy Chromaticity diagram**

Associated with the XYZ tristimulus values are the chromaticity coordinates:

\begin{align}
        x = X/(X+Y+Z)
\end{align}
\begin{align}
        y = Y/(X+Y+Z)
\end{align}
\begin{align}
        z = Z(X+Y+Z) = 1 - x - y
\end{align}

The XYZ value can be recovered using the Y value if stored as xyY:

\begin{align}
        X = xY/y
\end{align}
\begin{align}
        Z = (1 - x - y)Y/y
\end{align}

**Colour Spaces Conversions**

A set of formulas that define a relationship between a colour vector and the standard CIE XYZ colour space. The matrix to convert between these colour spaces can be calculated from solving the following set of linear equations for S<sub>r</sub>, S<sub>g</sub>, S<sub>b</sub>.

1. Get the primaries (x<sub>R</sub>, y<sub>R</sub>), (x<sub>G</sub>, y<sub>G</sub>), (x<sub>B</sub>, y<sub>B</sub>).
2. Get the white point (x<sub>W</sub>, y<sub>W</sub>) and maximum luminance Y<sub>W</sub>.
3. Compute the z chromaticity coordinate for each primary and white point.
4. Compute the XYZ tristimulus value for the whitepoint.
5. Solve the linear equations:

\begin{align}
        \left\{
        \begin{array}{cl}
        X_W = x_RS_R + x_GS_G + x_BS_B\\
        Y_W = y_RS_R + y_GS_G + y_BS_B\\
        Z_W = z_RS_R + z_GS_G + z_BS_B
        \end{array}
        \right.
    \end{align}

6. The conversion matrix from RGB to a new XYZ colour space is as follows:

\begin{align}
        \begin{pmatrix}
        x_RS_R & x_GS_G & x_BS_B\\
        y_RS_R & y_GS_G & y_BS_B\\
        z_RS_R & z_GS_G & z_BS_B
        \end{pmatrix}
    \end{align}

In [ ]:
# sRGB values

# primaries and whitepoint
x_R, x_G, x_B, x_W = 0.64, 0.3, 0.15, 0.3127
y_R, y_G, y_B, y_W = 0.33, 0.6, 0.06, 0.3290
Y_W = 1

# Gamma for simplified sRGB
gamma = 2.2

# z chromaticity
z_R = 1 - x_R - y_R
z_G = 1 - x_G - y_G
z_B = 1 - x_B - y_B
z_W = 1 - x_W - y_W

# XYZ tristimulus value for the white point
def xyY2XZ(x, y, Y):
  X = (x*Y)/y
  Z = (1 - x - y)*Y/y
  return X, Z
X_W, Z_W = xyY2XZ(x_W, y_W, Y_W)

# Recover the RGB2XYZ colour space conversion matrix
def get_RGB2XYZ(X_W, Y_W, Z_W, x_R, y_R, z_R, x_G, y_G, z_G, x_B, y_B, z_B):
  A = np.array([
      [x_R, x_G, x_B],
      [y_R, y_G, y_B],
      [z_R, z_G, z_B]
  ])
  B = np.array([X_W, Y_W, Z_W])

  # Solve the linear equation for B = AX
  X = np.linalg.solve(A, B)
  S_R, S_G, S_B = X
  print(f"S_R = {S_R}, S_G = {S_G}, S_B = {S_B}\n")

  M = np.array([
      [x_R * S_R, x_G * S_G, x_B * S_B],
      [y_R * S_R, y_G * S_G, y_B * S_B],
      [z_R * S_R, z_G * S_G, z_B * S_B]
  ])
  return M
sRGB2XYZ = get_RGB2XYZ(X_W, Y_W, Z_W, x_R, y_R, z_R, x_G, y_G, z_G, x_B, y_B, z_B)
XYZ2sRGB = np.linalg.inv(sRGB2XYZ)
print(f"sRGB to XYZ matrix:\n {sRGB2XYZ}\n")
print(f"XYZ to sRGB matrix:\n {XYZ2sRGB}")

S_R = 0.6443606238530615, S_G = 1.19194779794626, S_B = 1.2032052560122284

sRGB to XYZ matrix:
 [[0.4123908  0.35758434 0.18048079]
 [0.21263901 0.71516868 0.07219232]
 [0.01933082 0.11919478 0.95053215]]

XYZ to sRGB matrix:
 [[ 3.24096994 -1.53738318 -0.49861076]
 [-0.96924364  1.8759675   0.04155506]
 [ 0.05563008 -0.20397696  1.05697151]]


In [ ]:
xs = np.array([0.31, 0.31, 0.31, 0.19, 0.37, 0.45, 0.53, 0.30, 0.18, 0.38, 0.47, 0.20])
ys = np.array([0.32, 0.33, 0.33, 0.26, 0.25, 0.48, 0.33, 0.48, 0.14, 0.49, 0.31, 0.17])
Y = np.array([0.03, 0.20, 0.95, 0.21, 0.20, 0.61, 0.13, 0.25, 0.07, 0.45, 0.19, 0.12])
X = np.array([(x*Y)/y for x, y, Y in zip(xs, ys, Y)])
Z = np.array([(1 - x - y)*Y/y for x, y, Y in zip(xs, ys, Y)])

In [ ]:
XYZ = np.vstack((X,Y,Z))
B = np.array([XYZ2sRGB @ xyz for xyz in XYZ.T])
B[B < 0] = 0
B[B > 1] = 1
B

array([[0.03077363, 0.02955182, 0.03216114],
       [0.19264507, 0.20215974, 0.20026829],
       [0.91506407, 0.96025876, 0.95127436],
       [0.        , 0.26367159, 0.43524119],
       [0.5002728 , 0.10093012, 0.29699045],
       [0.87127036, 0.59375064, 0.00141393],
       [0.44931629, 0.04380128, 0.04339146],
       [0.06492328, 0.32230907, 0.07880928],
       [0.01454281, 0.05821452, 0.35009863],
       [0.37968192, 0.51090029, 0.05381359],
       [0.5742736 , 0.08283236, 0.1197901 ],
       [0.05132758, 0.10676148, 0.45341787]])

In [ ]:
rs = np.array([81, 515, 2483, 305, 790, 1835, 571, 431, 145, 1057, 854, 266])
gs = np.array([128, 804, 3890, 1173, 754, 1912, 327, 995, 471, 1616, 558, 746])
bs = np.array([110, 680, 3260, 1314, 896, 657, 236, 490, 818, 656, 468, 1145])

A = np.vstack((rs,gs,bs)).T
A = A/(np.power(2,12)-1)
A

array([[0.01978022, 0.03125763, 0.02686203],
       [0.12576313, 0.196337  , 0.16605617],
       [0.60634921, 0.94993895, 0.7960928 ],
       [0.07448107, 0.28644689, 0.32087912],
       [0.19291819, 0.18412698, 0.21880342],
       [0.44810745, 0.46691087, 0.16043956],
       [0.13943834, 0.07985348, 0.05763126],
       [0.10525031, 0.24297924, 0.11965812],
       [0.03540904, 0.11501832, 0.1997558 ],
       [0.25811966, 0.39462759, 0.16019536],
       [0.20854701, 0.13626374, 0.11428571],
       [0.06495726, 0.18217338, 0.27960928]])

In [ ]:
M = np.linalg.lstsq(A, B)[0].T
M

<ipython-input-82-4308c0b615e8>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  M = np.linalg.lstsq(A, B)[0].T


array([[ 3.47532895, -1.56895135,  0.4212012 ],
       [-0.39202336,  1.88719946, -0.75311668],
       [-0.09199522, -0.61359641,  1.99640353]])

In [ ]:
cc_RGB = np.array([M @ xyz for xyz in A])
cc_RGB[cc_RGB < 0] = 0
cc_RGB[cc_RGB > 1] = 1

cc_RGB

array([[0.03101539, 0.03100484, 0.03262819],
       [0.19896809, 0.19616532, 0.19947383],
       [0.95217019, 0.95547046, 0.95066211],
       [0.        , 0.26972467, 0.45798953],
       [0.47372816, 0.1070714 , 0.30609271],
       [0.89233769, 0.58465564, 0.        ],
       [0.38358223, 0.0526333 , 0.05322978],
       [0.03495696, 0.32717319, 0.08011218],
       [0.02673729, 0.05274191, 0.3249609 ],
       [0.3453737 , 0.52290625, 0.05392673],
       [0.55911556, 0.08933107, 0.12536414],
       [0.05769845, 0.10775433, 0.44045626]])

### Digital Cameras



**Mapping acquired RAW colours**

Colour Matching Functions, Spectral Reflectance *P<sub>j</sub>*, SPDs *I*, etc. are typically subsampled at regular intervals for digitisation. (e.g. Δλ = 10nm). Where *j* is a patch of a standard colour checker with known reflectance spectra.

The discrete version of calculating XYZ tristimulus values are:

\begin{align}
        X_j = \frac{1}{N} \sum_{i} I_i P_{j,i} x̄_i Δλ \\
        Y_j = \frac{1}{N} \sum_{i} I_i P_{j,i} ȳ_i Δλ \\
        Z_j = \frac{1}{N} \sum_{i} I_i P_{j,i} z̄_i Δλ \\
    \end{align}
\begin{align}
        N = ∑_i I_i ȳ_i Δλ
    \end{align}
Where:
- *j* is a patch of a standard colour checker with known reflectance spectra.
- *i* is an interval of the wavelength

**Mapping acquired colours**

The X<sub>j</sub>Y<sub>j</sub>Z<sub>j</sub> tristimulus values need to be converted to linear RGB values R<sub>j</sub>G<sub>j</sub>B<sub>j</sub>.

This requires the camera spectral sensitivities, css. The css describes the sensor output per unit incident light energy at each wavelength in the visible spectrum for RGB.

\begin{align}
        r_j = \sum_{i} I_i P_{j,i} css_{r,i} Δλ \\
        g_j = \sum_{i} I_i P_{j,i} css_{g,i} Δλ \\
        b_j = \sum_{i} I_i P_{j,i} css_{b,i} Δλ \\
    \end{align}

Solving the following linear equation will give the conversion matrix *M*:

\begin{align}
        \begin{pmatrix}
        r_1 & g_1 & b_1 \\
        ⋮ & ⋮ & ⋮ \\
        r_n & g_n & b_n
        \end{pmatrix} × M = \begin{pmatrix}
        R_1 & G_1 & B_1 \\
        ⋮ & ⋮ & ⋮ \\
        R_n & G_n & B_n
        \end{pmatrix}
    \end{align}

M can be used to map any *r<sub>c</sub>g<sub>c</sub>b<sub>c</sub>* camera space value to the corresponding linear sRGB colour *r<sub>m</sub>g<sub>m</sub>b<sub>m</sub>*:

\begin{align}
        \begin{pmatrix}
        r_m \\
        g_m \\
        b_m
        \end{pmatrix} = M × \begin{pmatrix}
        r_c \\
        g_c \\
        b_c
        \end{pmatrix}
    \end{align}

In [ ]:
# Digital XYZ acquisition:
def XYZvalues(SPD, P, CMF, interval):
  XYZ = np.empty((len(P[0]), 3)) # number of patches by XYZ (j, 3)
  N = SPD @ CMF[:, 1] * interval

  for ch in range(3):
    for j in range(len(P[0])):
      XYZ[j, ch] = (1/N) * sum([SPD[i] * P[i, j] * CMF[i, ch] * interval for i in range(len(SPD))])

  return XYZ

# linear RGB acquisition:
def linearRGBvalues(SPD, P, css, interval, rescale=0.95):
  rgb =  np.empty((len(P[0]), 3)) # number of patches by rgb (j, 3)

  for ch in range(3):
    for j in range(len(P[0])):
      rgb[j, ch] = sum([SPD[i] * P[i, j] * css[i, ch] * interval for i in range(len(SPD))])

  rgb = rgb/rgb.max()*rescale
  rgb[rgb < 0] = 0

  return rgb

# Find the camera space to sRGB colour space conversion matrix
def get_cRGB2sRGB(cRGB, sRGB):
  M = np.linalg.lstq(cRGB, sRGB)
  return M[0]


### Colour difference



**RGB colour difference**

The gamut of an RGB colour space can be arranged in a cube. Consider three colours:

- C<sub>1</sub> = [0.7, 0.5, 1]
- C<sub>2</sub> = [0.5, 0.5, 1]
- C<sub>3</sub> = [0.3, 0.5, 1]

The distance between two colours as the Euclidian distance, *d*, computed using their RGB component, d(C<sub>1</sub>, C<sub>2</sub>) = d(C<sub>2</sub>, C<sub>3</sub>)

However, in sRGB, they are not perceptually uniform.

**CIE 1976 (CIELAB)**

CIELAB is a colour space designed as a perceptually uniform colour space. The coordinates of a colour are indicated with (L, a, b)

The Euclidean distance in the CIELAB colour space is known as Delta E. Indicated as ΔE<sub>76</sub> or ΔE<sup>*</sup><sub>ab</sub>

\begin{align}
        ΔE_{76} = \sqrt{(ΔL)^2 + (Δa)^2 + (Δb)^2}
    \end{align}

**Conversion from CIE 1931 to CIELAB**

Given a reference white *W* with tristimulus values X<sub>W</sub>Y<sub>W</sub>Z<sub>W</sub>, the conversion of a tristimulus value XYZ in the CIE 1931 space to CIELAB is defined as follows:

\begin{align}
        L = 116f_y - 16 \\
        a = 500(f_x - f_y) \\
        b = 200(f_y - f_z) \\
    \end{align}
f<sub>x</sub>, f<sub>y</sub> and f<sub>z</sub> are defined as:
\begin{align}
        f_x = \left\{
          \begin{array}{cc}
          \sqrt[3]{x_r} & \text{if } x_r > ϵ \\
          \frac{kx_r+16}{116} & \text{otherwise}
          \end{array}
          \right. \\
        f_y = \left\{
          \begin{array}{cc}
          \sqrt[3]{y_r} & \text{if } y_r > ϵ \\
          \frac{ky_r+16}{116} & \text{otherwise}
          \end{array}
          \right. \\
        f_z = \left\{
          \begin{array}{cc}
          \sqrt[3]{z_r} & \text{if } z_r > ϵ \\
          \frac{kz_r+16}{116} & \text{otherwise}
          \end{array}
          \right. \\
    \end{align}
x<sub>r</sub>, y<sub>r</sub> and z<sub>r</sub> are defined as:
\begin{align}
        x_r = \frac{X}{X_W} \\
        y_r = \frac{Y}{Y_W} \\
        z_r = \frac{Z}{Z_W}
    \end{align}
Finally, the values of the constants are: ϵ = 216/24389 and *k* = 24389/27

In [ ]:

# Constants
epsilon = 216/24389
k = 24389/27

# CIELAB Euclidian Distance
def delta_E(L1, a1, b1, L2, a2, b2):
  L = L1 - L2
  a = a1 - a2
  b = b1 - b2
  return np.sqrt(np.sum(np.square([L, a, b])))

def cielab_L(f_y):
  return (116*f_y) - 16

def cielab_a(f_x, f_y):
  return 500*(f_x - f_y)

def cielab_b(f_y, f_z):
  return 200*(f_y - f_z)

def cielab_f_x(X, X_W):
  x_r = X/X_W
  if x_r > epsilon:
    return np.cbrt(x_r)
  else:
    return ((k*x_r) + 16)/116

def cielab_f_y(Y, Y_W):
  y_r = Y/Y_W
  if y_r > epsilon:
    return np.cbrt(y_r)
  else:
    return ((k*y_r) + 16)/116

def cielab_f_z(Z, Z_W):
  z_r = Z/Z_W
  if z_r > epsilon:
    return np.cbrt(z_r)
  else:
    return ((k*z_r) + 16)/116

def XYZ_to_CIELab(XYZ, XYZ_W):
  X = XYZ[0]
  Y = XYZ[1]
  Z = XYZ[2]
  X_W = XYZ_W[0]
  Y_W = XYZ_W[1]
  Z_W = XYZ_W[2]

  f_x = cielab_f_x(X, X_W)
  f_y = cielab_f_y(Y, Y_W)
  f_z = cielab_f_z(Z, Z_W)

  L = cielab_L(f_y)
  a = cielab_a(f_x, f_y)
  b = cielab_b(f_y, f_z)

  return L,a,b


In [ ]:
reference_white = np.array([X_W, Y_W, Z_W])
reference_white

array([0.95045593, 1.        , 1.08905775])

In [ ]:
sRGB_Lab = np.array([XYZ_to_CIELab(rgb, reference_white) for rgb in B])
sRGB_Lab

array([[ 1.98635063e+01,  4.77822472e+00,  1.44141883e-02],
       [ 5.20805222e+01,  2.54901763e-01,  3.64792873e+00],
       [ 9.84425207e+01,  4.28486728e-01,  6.13212334e+00],
       [ 5.83839270e+01, -2.51654858e+02, -1.90700330e+01],
       [ 3.80088489e+01,  1.70905924e+02, -3.65774760e+01],
       [ 8.14973218e+01,  6.54628624e+01,  1.38490631e+02],
       [ 2.48902952e+01,  2.13251061e+02,  2.19100673e+00],
       [ 6.35332690e+01, -1.38425889e+02,  5.37827472e+01],
       [ 2.89574589e+01, -6.96525118e+01, -5.94942684e+01],
       [ 7.67335117e+01, -3.14719826e+01,  8.64942183e+01],
       [ 3.45659295e+01,  2.04743453e+02, -8.64409415e+00],
       [ 3.90295788e+01, -4.82046719e+01, -5.44624786e+01]])

In [ ]:
cc_Lab = np.array([XYZ_to_CIELab(rgb, reference_white) for rgb in cc_RGB])
cc_Lab

array([[  20.44191045,    2.7013237 ,    0.71384997],
       [  51.40085436,    6.36332386,    2.62667671],
       [  98.25198238,    7.83498211,    5.8446335 ],
       [  58.94883381, -254.08980092,  -20.6191757 ],
       [  39.08277622,  159.00658212,  -36.03755678],
       [  80.99694092,   71.50336343,  139.64989814],
       [  27.47218954,  182.11793807,    1.82727037],
       [  63.93136449, -178.25280268,   54.01234025],
       [  27.50207127,  -35.4430443 ,  -58.64266338],
       [  77.45429539,  -46.02222229,   87.6855484 ],
       [  35.85518336,  195.43403629,   -7.88509256],
       [  39.19963858,  -41.42125195,  -52.73244852]])

In [ ]:
difference_Lab = sRGB_Lab - cc_Lab
delta_Es = [np.sqrt(np.sum(np.square(difference))) for difference in difference_Lab]
delta_Es, np.mean(delta_Es)

([2.2665567606086454,
  6.230387197933724,
  7.414521498090755,
  2.940733196084265,
  11.959898030162902,
  6.171056128820663,
  31.24211632096787,
  39.82956474332243,
  34.2510007641337,
  14.616712086580778,
  9.428864943021983,
  7.002621674218139],
 14.44616944532882)

# Topic 2: Multiview Geometry and Local Features

## Multiview Geometry

### Pose Estimation

Rotation matrix *R* and translation vector *t* convert world to camera coordinates. [R t] define the pose of the camera.

Position of the camera centre is given by:
\begin{align}
        c = -R^⊤t
    \end{align}

The pose of the camera can be estimated via linear least squares.
Given *n* pairs of 2D-3D corresponding points:

\begin{align}
        \begin{array}{cc}
        X_1 = (x_1,y_1) & \iff & W_1 = (u_1,v_1,w_1)\\
        & ⋮ & \\
        X_n = (x_n,y_n) & \iff & W_n = (u_n,v_n,w_n)\\
        \end{array}
    \end{align}

with the intrinsic parameter matrix *K*, for each pair *i* we have the perspective projection equation:

\begin{align}
        λ_i \begin{pmatrix}
        x_i \\ y_i \\ 1
        \end{pmatrix} = K \begin{pmatrix}
        r_{1,1} & r_{1,2} & r_{1,3} & t_x \\
        r_{2,1} & r_{2,2} & r_{2,3} & t_y \\
        r_{3,1} & r_{3,2} & r_{3,3} & t_z \\
        \end{pmatrix} \begin{pmatrix}
        u_i \\ v_i \\ w_i \\ 1
        \end{pmatrix}
    \end{align}

1. Multiply both sides by K<sup>-1</sup> to remove K:

\begin{align}
        λ_i \begin{pmatrix}
        x'_i \\ y'_i \\ 1
        \end{pmatrix} =\begin{pmatrix}
        r_{1,1} & r_{1,2} & r_{1,3} & t_x \\
        r_{2,1} & r_{2,2} & r_{2,3} & t_y \\
        r_{3,1} & r_{3,2} & r_{3,3} & t_z \\
        \end{pmatrix} \begin{pmatrix}
        u_i \\ v_i \\ w_i \\ 1
        \end{pmatrix}
    \end{align}

2. λ is equivalent to the following equation from the matrix:

\begin{align}
        λ_i = r_{3,1}u_i + r_{3,2}v_i + r_{3,3}w_i + t_z
    \end{align}

3. Substitute for λ<sub>i</sub> to get:

\begin{align}
        \begin{pmatrix}
        (r_{3,1}u_i + r_{3,2}v_i + r_{3,3}w_i + t_z) x'_i \\
        (r_{3,1}u_i + r_{3,2}v_i + r_{3,3}w_i + t_z) y'_i \\
        \end{pmatrix} =\begin{pmatrix}
        r_{1,1} & r_{1,2} & r_{1,3} & t_x \\
        r_{2,1} & r_{2,2} & r_{2,3} & t_y \\
        \end{pmatrix} \begin{pmatrix}
        u_i \\ v_i \\ w_i \\ 1
        \end{pmatrix}
    \end{align}

4. Rearrange for homogeneous linear equation in unknown pose:

\begin{align}
        \begin{pmatrix}
        u_i & v_i & w_i & 1 & 0 & 0 & 0 & 0 & -u_ix'_i & -v_ix'_i & -w_ix'_i & -x'_i \\
        0 & 0 & 0 & 0 & u_i & v_i & w_i & 1 & -u_iy'_i & -v_iy'_i & -w_iy'_i & -y'_i \\
        ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ \\
        u_n & v_n & w_n & 1 & 0 & 0 & 0 & 0 & -u_nx'_n & -v_nx'_n & -w_nx'_n & -x'_n \\
        0 & 0 & 0 & 0 & u_n & v_n & w_n & 1 & -u_ny'_n & -v_ny'_n & -w_ny'_n & -y'_n \\
        \end{pmatrix} × \begin{pmatrix}
        r_{1,1} \\ r_{1,2} \\ r_{1,3} \\ t_x \\
        r_{2,1} \\ r_{2,2} \\ r_{2,3} \\ t_y \\
        r_{3,1} \\ r_{3,2} \\ r_{3,3} \\ t_z \\
        \end{pmatrix} = 0
    \end{align}

5. Solve the linear problem with SVD

6. Reparameterise R to a valid rotation matrix.

In [ ]:
# Pose Estimation -> returns pose R, t
def pose_estimation(X, W, K):
  npnts = X.shape[0]
  A = np.empty((npnts*2, 12))
  for i in range(npnts):
    xy_prime = np.linalg.inv(K) @ np.vstack((X[i,0], X[i,1], 1))
    x, y, _ = xy_prime
    x, y = x[0], y[0]
    u, v, w = W[i]

    A[2*i, :] = [u, v ,w ,1 ,0 ,0 ,0 ,0 , -u*x, -v*x, -w*x, -x]
    A[2*i+1, :] = [0, 0, 0, 0, u, v, w, 1, -u*y, -v*y, -w*y, -y]

  Rt = np.linalg.svd(A)
  Rt = Rt[-1][-1, :].reshape((3, 4))
  R = Rt[:, :3]
  t = Rt[:, 3]
  rot = Rotation.from_matrix(R)
  return rot.as_matrix, t

### 3D reconstruction



**Multiview Reconstruction Problem**

- *n* cameras are viewing the same world point *W* = (u,v,w).
- *i*<sup>th</sup> camera has known intrinsic matrix *K<sub>i</sub>* and pose *R<sub>i</sub>*, *t<sub>i</sub>*.
- Projection of *W* into *i*<sup>th</sup> is X<sub>i</sub> = (x<sub>i</sub>,y<sub>i</sub>)
- Find (best fit) *W*

For each camera *i* for a point *W*:

\begin{align}
        λ_i \begin{pmatrix}
        x_i \\ y_i \\ 1
        \end{pmatrix} = K_i \begin{pmatrix}
        R_i & t_i
        \end{pmatrix} \begin{pmatrix}
        u \\ v \\ w \\ 1
        \end{pmatrix}
    \end{align}

1. Multiply both sides by K<sup>-1</sup> to remove K:

\begin{align}
        λ_i \begin{pmatrix}
        x'_i \\ y'_i \\ 1
        \end{pmatrix} = \begin{pmatrix}
        R_i & t_i
        \end{pmatrix} \begin{pmatrix}
        u \\ v \\ w \\ 1
        \end{pmatrix}
    \end{align}

2. λ is equivalent to the following equation from the matrix:

\begin{align}
        λ_i = r_{i,3,1}u + r_{i,3,2}v + r_{i,3,3}w + t_{i,z}
    \end{align}

3. Substitute for λ_i to get:

\begin{align}
        \begin{pmatrix}
        (r_{i,3,1}u + r_{i,3,2}v + r_{i,3,3}w + t_{i,z})x'_i \\
        (r_{i,3,1}u + r_{i,3,2}v + r_{i,3,3}w + t_{i,z})y'_i
        \end{pmatrix} = \begin{pmatrix}
        r_{i,1,1} & r_{i,1,2} & r_{i,1,3} & t_{i,x} \\
        r_{i,2,1} & r_{i,2,2} & r_{i,2,3} & t_{i,y} \\
        \end{pmatrix} \begin{pmatrix}
        u \\ v \\ w \\ 1
        \end{pmatrix}
    \end{align}

4. Rearrange into two linear equations with unknown W = [u, v, w]<sup>⊤</sup> and stack for each camera:

\begin{align}
        \begin{pmatrix}
        r_{i,3,1}x'_i-r_{i,1,1} &
        r_{i,3,2}x'_i-r_{i,1,2} &
        r_{i,3,3}x'_i-r_{i,1,3} \\
        r_{i,3,1}y'_i-r_{i,2,1} &
        r_{i,3,2}y'_i-r_{i,2,2} &
        r_{i,3,3}y'_i-r_{i,2,3} \\
        ⋮ & ⋮ & ⋮ \\
        r_{n,3,1}x'_n-r_{n,1,1} &
        r_{n,3,2}x'_n-r_{n,1,2} &
        r_{n,3,3}x'_n-r_{n,1,3} \\
        r_{n,3,1}y'_n-r_{n,2,1} &
        r_{n,3,2}y'_n-r_{n,2,2} &
        r_{n,3,3}y'_n-r_{n,2,3} \\
        \end{pmatrix} \begin{pmatrix}
        u \\ v \\ w
        \end{pmatrix} = \begin{pmatrix}
        t_{i,x}-t_{i,z}x'_i \\
        t_{i,y}-t_{i,z}y'_i \\
        ⋮ \\
        t_{n,x}-t_{n,z}x'_n \\
        t_{n,y}-t_{n,z}y'_n \\
        \end{pmatrix}
    \end{align}

In [ ]:
# 3D reconstruction of point W
def reconstruct_point(Xs, Ks, Rs, ts):
  ncameras = Xs.shape[0]
  A = np.empty((ncameras*2, 3))
  B = np.empty((ncameras*2, 1))

  for i in range(ncameras):
    xy_prime = np.linalg.inv(Ks[i]) @ np.vstack((Xs[i,0], Xs[i,1], 1))
    x, y, _ = xy_prime
    x, y = x[0], y[0]
    r = Rs[i]
    t = ts[i]

    A[i*2, :] = [r[2,0]*x-r[0,0], r[2,1]*x-r[0,1], r[2,2]*x-r[0,2]]
    A[i*2+1,:] = [r[2,0]*y-r[1,0], r[2,1]*y-r[1,1], r[2,2]*y-r[1,2]]
    B[i*2] = [t[0]-t[2]*x]
    B[i*2+1] = [t[1]-t[2]*y]
  W = np.linalg.lstsq(A, B)
  return W[-1]

### Planes

**Looking at Planes**

Perspective projection of a world point *W* on a plane:

\begin{align}
        W = \begin{pmatrix} u \\ v \\ 0 \end{pmatrix}
    \end{align}

*w* can be removed from the world coordinate to give a simplified form of perspective projection:

\begin{align}
        λ \begin{pmatrix} x \\ y \\ 1 \end{pmatrix} =
        K \begin{pmatrix}
        r_{1,1} & r_{1,2} & t_x \\
        r_{2,1} & r_{2,2} & t_y \\
        r_{3,1} & r_{3,2} & t_z \\
        \end{pmatrix} \begin{pmatrix}
        u \\ v \\ 1
        \end{pmatrix}
    \end{align}

**Homographies**

Multiply by K<sup>-1</sup> and truncate [R t] matrices together gives a 3x3 matrix that translates between a point on a 3D plane and a point in a 2D image. This is the homography matrix, *H*:

\begin{align}
        λ \begin{pmatrix} x \\ y \\ 1 \end{pmatrix} = \begin{pmatrix}
        h_{1,1} & h_{1,2} & h_{1,3} \\
        h_{2,1} & h_{2,2} & h_{2,3} \\
        h_{3,1} & h_{3,2} & h_{3,3} \\
        \end{pmatrix}
        \begin{pmatrix} u \\ v \\ 1 \end{pmatrix}
        = H \begin{pmatrix} u \\ v \\ 1 \end{pmatrix}
    \end{align}

Divide by λ to get the Cartesian coordinates:

\begin{align}
        λ  = h_{3,1}u + h_{3,2}v + h{3,3} \\[1em]
        x = \frac{h_{1,1}u+h_{1,2}v+h_{1,3}}{λ} \\
        y = \frac{h_{2,1}u+h_{2,2}v+h_{2,3}}{λ}
    \end{align}

Homographies define a one-to-one map from plane to image:

\begin{align}
        λ \begin{pmatrix} x \\ y \\ 1 \end{pmatrix}
        = H \begin{pmatrix} u \\ v \\ 1 \end{pmatrix}
    \end{align}

or image to plane:

\begin{align}
        λ \begin{pmatrix} u \\ v \\ 1 \end{pmatrix}
        = H^{-1} \begin{pmatrix} x \\ y \\ 1 \end{pmatrix}
    \end{align}

3D-2D projection reduces to 2D-2D mapping. H only defined up to an unknown scale.

**Computing a Homography**

Given *n* pairs of corresponding points:

\begin{array}{cc}
        (x_i,y_i) & \iff & (u_i,v_i)\\
        & ⋮ & \\
        (x_n,y_n) & \iff & (u_n,v_n)
    \end{array}

Each corresponding point pair gives the equation:

\begin{align}
        λ \begin{pmatrix} x_i \\ y_i \\ 1 \end{pmatrix} = \begin{pmatrix}
        h_{1,1} & h_{1,2} & h_{1,3} \\
        h_{2,1} & h_{2,2} & h_{2,3} \\
        h_{3,1} & h_{3,2} & h_{3,3} \\
        \end{pmatrix} \begin{pmatrix} u_i \\ v_i \\ 1 \end{pmatrix}
    \end{align}

1. Using the Direct Linear Transform trick on the equation:

\begin{align}
        \begin{pmatrix} x_i \\ y_i \\ 1 \end{pmatrix} × \begin{pmatrix}
        h_{1,1} & h_{1,2} & h_{1,3} \\
        h_{2,1} & h_{2,2} & h_{2,3} \\
        h_{3,1} & h_{3,2} & h_{3,3} \\
        \end{pmatrix} \begin{pmatrix} u_i \\ v_i \\ 1 \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix}
    \end{align}

2. Rearrange the equation into a linear least squares problem, solve for unknown H with SVD:

\begin{align}
        \begin{pmatrix}
        0 & 0 & 0 & -u_i & -v_i & -1 & y_iu_i & y_iv_i & y_i \\
        u_i & v_i & 1 & 0 & 0 & 0 & -x_iu_i & -x_iv_i & -x_i \\
        ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ \\
        0 & 0 & 0 & -u_n & -v_n & -1 & y_nu_n & y_nv_n & y_n \\
        u_n & v_n & 1 & 0 & 0 & 0 & -x_nu_n & -x_nv_n & -x_n \\
        \end{pmatrix} \begin{pmatrix}
        h_{1,1} \\ h_{1,2} \\ h_{1,3} \\
        h_{2,1} \\ h_{2,2} \\ h_{2,3} \\
        h_{3,1} \\ h_{3,2} \\ h_{3,3} \\
        \end{pmatrix} = 0
    \end{align}

In [ ]:
# Get Homography
def get_homography(Xs, Ws):
  npnts = Xs.shape[0]
  A = np.empty((npnts*2, 9))
  for i in range(npnts):
    x, y = Xs[i,:]
    u, v = Ws[i,:]

    A[i*2,:] = [0, 0, 0, -u, -v, -1, y*u, y*v, y]
    A[i*2+1,:] = [u, v, 1, 0, 0, 0, -x*u, -x*v, -x]

  H = np.linalg.svd(A)
  return H[-1][-1,:].reshape((3, 3))

### Two View Geometry

**Fundamental Matrix**

The relationship between points in one image to another in captured by the Fundamental Matrix which represents a line.

\begin{align}
        \begin{pmatrix}
        x' & y' & 1
        \end{pmatrix} F \begin{pmatrix}
        x \\ y \\ 1
        \end{pmatrix} = 0
    \end{align}

Where:

\begin{align}
        F = K_2^{-⊤}EK_1^{-1}=K_2^{-⊤}[t] × RK_1^{-1}
    \end{align}
- F : Fundamental Matrix
- K<sub>1</sub> : Intrinsic Matrix of Camera 1
- K<sub>2</sub> : Intrinsic Matrix of Camera 2
- E = [t] × R : Essential Matrix, the cross product of t and R
  - t : relative translation from Camera 1 to Camera 2
  - R : relative rotation from Camera 1 to Camera 2

**Fundamental Matrix Estimation**

Given a set of point correspondences:

\begin{array}{cc}
  (x_{1,i},y_{1,i}) & \iff & (u_i, v_i) & \iff & (x_{2,i},y_{2,i}) \\
  ⋮ & & ⋮ & & ⋮ \\
  (x_{1,n},y_{1,n}) & \iff & (u_n, v_n) & \iff & (x_{2,n},y_{2,n}) \\
    \end{array}

Each corresponding point gives the equation:

\begin{align}
        \begin{pmatrix}
        x_{2,i} & y_{2,i} & 1
        \end{pmatrix} \begin{pmatrix}
        f_{1,1} & f_{1,2} & f_{1,3} \\
        f_{2,1} & f_{2,2} & f_{2,3} \\
        f_{3,1} & f_{3,2} & f_{3,3} \\
        \end{pmatrix} \begin{pmatrix}
        x_{1,i} \\ y_{1,i} \\ 1
        \end{pmatrix} = 0
    \end{align}

1. Rearrange to form a linear least squares problem and solve for F with SVD:

\begin{align}
        Af = \begin{pmatrix}
        x_{2,i}x_{1,i} & x_{2,i}y_{1,i} & x_{2,i} & y_{2,i}x_{1,i} & y_{2,i}y_{1,i} & y_{2,i} & x_{1,i} & y_{1,i} & 1 \\
        ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮ & ⋮\\
        x_{2,n}x_{1,n} & x_{2,n}y_{1,n} & x_{2,n} & y_{2,n}x_{1,n} & y_{2,n}y_{1,n} & y_{2,n} & x_{1,n} & y_{1,n} & 1 \\
        \end{pmatrix} \begin{pmatrix}
        f_{1,1} \\ f_{1,2} \\ f_{1,3} \\
        f_{2,1} \\ f_{2,2} \\ f_{2,3} \\
        f_{3,1} \\ f_{3,2} \\ f_{3,3} \\
        \end{pmatrix} = 0
    \end{align}

In [ ]:
# Fundamental Matrix Estimation
def compute_F(X1, X2):
  npnts = X1.shape[0]
  A = np.empty((npnts, 9))

  for i in range(npnts):
    x1, y1 = X1[i,:]
    x2, y2 = X2[i,:]
    A[i,:] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1]

  f = np.linalg.svd(A)
  f = f[-1][-1,:].reshape((3,3))

  return f / f[2,2]

## Local Features, Corners and Descriptors

### SIFT

**SIFT:** Scale-invariant Feature Transform is an algorithm for point based feature detection, description and matching.

**Scale-space**

An algorithm that searches over all scales and image locations to identify potential interest points.

The scale-space of an image I(x,y) is defined as a function L(x,y,σ):

\begin{align}
        L(x,y,σ) = G(x,y,σ) * I(x,y)\\[1em]
        G(x,y,σ) = \frac{1}{2πσ^2}e^{-(x^2+y^2)/2σ^2}
    \end{align}

Implemented efficiently by using a difference-of-Gaussian D(x,y,σ):

\begin{array}{cl}
        D(x,y,σ) & = (G(x,y,kσ)-G(x,y,σ))*I(x,y)\\
        & = L(x,y,kσ) - L(x,y,σ)
\end{array}



# Topic 3: Surface Reflectance and Rendering

## Surface Reflectance and BRDF models

**BRDF: Bidirectional Reflectance Distribution Function**

A function to model the spectral distribution of reflected light and the directional distribution of the light.

\begin{align}
        f_r(v_i, v_r) = \frac{dL_r(v_r)}{dEi(v_i)} = \frac{dL_r(v_r)}{L_i(v_i)cosθ_idω_i}
    \end{align}

- L<sub>r</sub> : outgoing radiance
- L<sub>i</sub> : incoming radiance
- E<sub>i</sub> : incoming irradiance
- v<sub>r</sub>, v<sub>i</sub> : the reflected and incident direction of the ray of light.
- θ<sub>r</sub>, θ<sub>i</sub> : the viewing angle and mirror incident angle of light respect to the normal
- dω<sub>r</sub>, dω<sub>i</sub> : the solid angle of the reflected and incident light.
- The unit of measurement for a BRDF is inverse steradian (1/sr)

**Halfway vector:** The halfway direction between the viewing direction and mirror direction of incoming light.

\begin{align}
        h = \frac{(v_i+v_r)}{||v_i+v_r||}
    \end{align}

![picture](https://onlinelibrary.wiley.com/cms/asset/59d44032-42fc-4493-90b5-23c6afd7189b/cgf12867-fig-0002-m.png)

**Coordinate System**

- n : the normal to the surface
- t : the tangent vector of the incoming light to the plane
- b : the bi-tangent vector
- θ is measured respect to the normal n
- Φ is measured respect to the tangent, counter-clockwise
- z-axis respect to n
- y-axis respect to b
- x-axis respect to t

A coordinate can be written as:

\begin{array}{cc}
        \vec{v} = (θ_v,Φ_v) & \vec{v} = (x_v, y_v, z_v)
    \end{array}

**Fresnel Reflectance**

The Fresnel effect predicts the fraction of power which is reflected and transmitted (refracted) as a function of the viewing angle.

For unpolarised light, the Fresnel reflectance at the interface between the air and a surface is given by Maxwell's Equation:

\begin{align}
        𝓕_M(ƞ, θ_i, θ_t) = \frac{1}{2} \left[ \left(\frac{ƞcosθ_i - cosθ_t}{ƞcosθ_i + cosθ_t}\right)^2 + \left(\frac{cosθ_i - ƞcosθ_t}{cosθ_i + ƞcosθ_t}\right)^2 \right]
    \end{align}

- ƞ is the index of refraction of the surface
- θ<sub>i</sub> is the angle of incidence
- θ<sub>t</sub> is the angle of transmission (refraction)

In computer graphics, it is very common to use Schlick's approximation of the Fresnel reflectance:

\begin{array}{cl}
        𝓕(θ) = 𝓕(0) + (1 - 𝓕(0))(1-cosθ)^5\\
        𝓕(0) = \left(\frac{ƞ_1-ƞ_2}{ƞ_1+ƞ_2}\right)^2
    \end{array}
- 𝓕(0) is the reflectance at normal incidence
- ƞ<sub>1</sub> is the index of refraction of the first medium
- ƞ<sub>2</sub> is the index of refraction of the material

In [ ]:
def halfway_vector(v_i, v_r):
  return (v_i + v_r)/np.linalg.norm(v_i + v_r)

# Convert between the BRDF coordinate systems
def vector_coordinates(theta, phi):
  z = np.cos(np.deg2rad(theta))

  xy = np.sin(np.deg2rad(theta))
  x = xy * np.cos(np.deg2rad(phi))
  y = xy * np.sin(np.deg2rad(phi))

  return x,y,z

def vector_angles(x, y, z):
  theta = np.arccos(z)
  phi = np.arctan2(y, x)

  return np.degrees(theta), np.degrees(phi)

# Fresnel Effect
def maxwell(ior, theta_i, theta_t):
  return 0.5(np.square((ior*np.cos(theta_i) - np.cos(theta_t))/(ior*np.cos(theta_i) + np.cos(theta_t))) +
             np.square((np.cos(theta_i) - ior*np.cos(theta_t))/(np.cos(theta_i) + ior*np.cos(theta_t))))

def schlick(theta, n_2, n_1 = 1):
  f_0 = np.square((n_1 - n_2)/(n_1 + n_2))
  return f_0 + (1 - f_0)*np.power(1 - np.cos(np.deg2rad(theta)), 5)

<>:22: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:22: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<ipython-input-37-f790b7be2fe0>:22: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
  return 0.5(np.square((ior*np.cos(theta_i) - np.cos(theta_t))/(ior*np.cos(theta_i) + np.cos(theta_t))) +


### Phenomenological BRDF models

**Phong Model**

Model for non-Lambertian surfaces. The specular reflection is modelled by the means of:

- k<sub>s</sub> a specular constant
- v<sub>r</sub>, v<sub>i</sub> : the direction of the incident ray and viewing angle.
- r<sub>vi</sub> : the mirror direction of the incident ray.
- m : an exponent that controls the width of the specular lobe.

\begin{align}
        f_r(\vec{v_i}, \vec{v_r}) = k_s(\vec{v_r}·\vec{r_{v_i}})^m
    \end{align}

In [ ]:
def phong(k, theta_i, phi_i, theta_r, phi_r, m):
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  n = np.array([0,0,1])
  # calculate mirror vector
  r_vi = v_i - 2 * np.dot(v_i, n) * n

  return k * np.power(np.dot(v_r, r_vi), m)

**Blinn-Phong**

A simple correction to the phong model. The mirror direction no longer needs to be calculated at each position of the scene.

Blinn: when an orthographic camera is used, the halfway vector can be approximately considered to be constant accross the scene.

\begin{align}
        f_r(\vec{v_i}, \vec{v_r}) = k_s(\vec{v_r}·\vec{h})^m
    \end{align}

In [ ]:
def blinn_phong(k, theta_i, phi_i, theta_r, phi_r, m):
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  h = halfway_vector(v_i, v_r)
  return k * np.power(np.dot(v_r, h), m)

**Ward model**

Combines specular and diffuse components of reflectance. There are four parameters:

- ρ<sub>d</sub> and ρ<sub>s</sub> : diffuse and specular albedos
- α<sub>x</sub> and α<sub>y</sub> : roughness - control the width of the Gaussian love in the two principle directions on anisotropy

\begin{align}
        f_r(\vec{v_i}, \vec{v_r}) = \frac{ρ_d}{π} + \frac{\rho_s}{4\sqrt{cos\theta_rcos\theta_i}} ⋅ \frac{e^{-\frac{((\vec{h}⋅\vec{x})/a_x)^2+((\vec{h}⋅\vec{y})/a_y)^2}{(\vec{h}⋅\vec{n})^2}}}{\pi\alpha_x\alpha_y}
\end{align}

In [ ]:
def ward(theta_i, phi_i, theta_r, phi_r, rho_d, rho_s, alpha_x, alpha_y, x = np.array([1, 0, 0]), y = np.array([0, 1, 0]), n = np.array([0, 0, 1]), ):
  """
    Compute the value of the Ward BRDF model, given the surface parameters and lighting and viewing directions.

    Args:
        theta_i: Polar angle of the incident light.
        phi_i: Azimuthal angle of the incident light.
        theta_r: Polar angle of the reflection.
        phi_r: Azimuthal angle of the reflection.
        x, y, n: Axis of the coordinate system.
        rho_d: Diffuse albedo.
        rho_s: Specular albedo.
        alpha_x, alpha_y: Surface roughness parameters.

    Returns:
        BRDF value for the input parameters.
    """
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)

  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  h = halfway_vector(v_i, v_r)

  specular_denominator =  4 * np.pi * alpha_x * alpha_y * np.sqrt(np.cos(np.deg2rad(theta_i)) * np.cos(np.deg2rad(theta_r)))
  exponent = - (np.power((np.dot(h, x)/alpha_x), 2) + np.power((np.dot(h, y)/alpha_y), 2))/(np.power(np.dot(h, n), 2))
  specular = (rho_s / specular_denominator) * np.exp(exponent)
  diffuse = rho_d / np.pi

  return diffuse + specular

**Ward - energy conservation**

The Ward specular denominator is replaced by GMD's specular denominator which can model energy conservation.

\begin{align}
        4\sqrt{cos\theta_rcos\theta_i} ⇒ \frac{2(1+cos\theta_icos\theta_r+sin\theta_rcos(\Phi_r-\Phi_i))}{(cos\theta_i+cos\theta_r)^4}
    \end{align}

In [ ]:
def ward_energy_conservation(theta_i, phi_i, theta_r, phi_r, rho_d, rho_s, alpha_x, alpha_y, x = np.array([1, 0, 0]), y = np.array([0, 1, 0]), n = np.array([0, 0, 1]), ):
  """
    Compute the value of the Ward BRDF model, given the surface parameters and lighting and viewing directions.

    Args:
        theta_i: Polar angle of the incident light.
        phi_i: Azimuthal angle of the incident light.
        theta_r: Polar angle of the reflection.
        phi_r: Azimuthal angle of the reflection.
        x, y, n: Axis of the coordinate system.
        rho_d: Diffuse albedo.
        rho_s: Specular albedo.
        alpha_x, alpha_y: Surface roughness parameters.

    Returns:
        BRDF value for the input parameters.
    """
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)

  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  h = halfway_vector(v_i, v_r)

  GMD = (2(1 + np.cos(np.deg2rad(theta_i))*np.cos(np.deg2rad(theta_r)) + np.sin(np.deg2rad(theta_i))*np.sin(np.deg2rad(theta_r))*np.cos(np.deg2rad(phi_r-phi_i))))/np.power((np.cos(np.deg2rad(theta_i))+np.cos(np.deg2rad(theta_r))), 4)
  specular_denominator =  np.pi * alpha_x * alpha_y * GMD
  exponent = - (np.power((np.dot(h, x)/alpha_x), 2) + np.power((np.dot(h, y)/alpha_y), 2))/(np.power(np.dot(h, n), 2))
  specular = (rho_s / specular_denominator) * np.exp(exponent)
  diffuse = rho_d / np.pi

  return diffuse + specular

<>:26: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:26: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<ipython-input-41-f142cc350c82>:26: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  GMD = (2(1 + np.cos(np.deg2rad(theta_i))*np.cos(np.deg2rad(theta_r)) + np.sin(np.deg2rad(theta_i))*np.sin(np.deg2rad(theta_r))*np.cos(np.deg2rad(phi_r-phi_i))))/np.power((np.cos(np.deg2rad(theta_i))+np.cos(np.deg2rad(theta_r))), 4)


### Physically-based Models

**Microfacets theory**

This assumes that the surface is rough at a fine scale and can be described by a collection of planar micro facets.

The most common model:

\begin{align}
        f_r(\vec{v_i},\vec{v_r}) = \frac{D(...)⋅G(...)⋅𝓕(...)}{k(\vec{v_i},\vec{v_r})}
    \end{align}

- D is the distribution of the size and direction of the microfacets. Only the fraction of the facets oriented in the direction of h contributes to the reflection.
- G is the geometrical attenuation term (models shadowing and masking)
- 𝓕 is the Fresnel term
- k is some normalisation constant; often k=4

**Microfacets distribution**

The distribution D must be normalised.

1. Take into account a microfacet with the normal direction h and area dA<sub>h</sub>. It's projected area on the macro-surface respect to the normal n is dA:

\begin{align}
        dA = (n⋅h)dA_h
    \end{align}

2. The sum of all microfacet areas ∑dA<sub>h</sub> must be equal to the macro-surface area A.

3. Therefore, if A=1, the following integral over the hemisphere must be equal to 1:

\begin{align}
        ∫_{Ω^2} D(h)(n⋅h)dω_h = 1
    \end{align}
where h∈Ω<sup>2</sup>, dω<sub>h</sub> is a small solid angle around h.

The above integral represents the probability of having a microfacet normal within dω<sub>h</sub> from h.

In [ ]:
def projected_area(n, h, dA_h):
  return dA_h*np.dot(n, h)

**Cook-Torrance model**

The model takes into account both specular and diffuse reflections, the latter modelled as Lambertian reflection.

The specular component assumed that only a fraction of the facets oriented in the direction h contributes to the final reflection.

These factors are modelled respectively through the functions D, G and F:

\begin{align}
        f_r(\vec{v_i},\vec{v_r}) = \frac{ρ_d}{\pi} +
        \frac{D(\vec{h})⋅G(\vec{v_i},\vec{v_r})⋅F(θ_r)}{πcosθ_rcosθ_i}
    \end{align}

**Cook-Torrance: microfacets distribution**

Beckmann distribution:

\begin{align}
        D(\vec{h}) = \frac{ e^{ -\left(\frac{tan\theta_h}{σ}\right)^2 } }{ 4σ^2(cos\theta_h)^4 }
    \end{align}

- σ : Root Mean Square slope of the microfacets.

**Cook-Torrance: geometric attenuation**

- Case 1 - micro-surface fully illuminated and visible:

\begin{align}
        G = 1
    \end{align}

- Case 2 - partial masking:

\begin{align}
        G(\vec{v_i},\vec{v_r},\vec{h}, \vec{n}) =
        \frac{2(\vec{h}⋅\vec{n})(\vec{n}⋅\vec{v_r})}
        {\vec{v_r}⋅\vec{h}}
    \end{align}

- Case 3 - partial shadowing:

\begin{align}
        G(\vec{v_i},\vec{v_r},\vec{h}, \vec{n}) =
        \frac{2(\vec{h}⋅\vec{n})(\vec{n}⋅\vec{v_i})}
        {\vec{v_r}⋅\vec{h}}
    \end{align}

Overall:

\begin{align}
        G(\vec{v_i},\vec{v_r},\vec{h}, \vec{n}) = min\left(1, \frac{2(\vec{h}⋅\vec{n})(\vec{n}⋅\vec{v_r})}
        {\vec{v_r}⋅\vec{h}},
        \frac{2(\vec{h}⋅\vec{n})(\vec{n}⋅\vec{v_i})}
        {\vec{v_r}⋅\vec{h}} \right)
    \end{align}

**Cook-Torrance: Fresnel term**

The Fresnel reflectance of the polished microfacets is approximated with the expression:

\begin{array}{cr}
        𝓕 = \frac{(g-c)^2}{2(g+c)^2} \left(
          1 + \frac{(c(g+c)-1)^2}{(c(g-c)-1)^2}
          \right)\\
        c = \vec{v_r}⋅\vec{h}=cosθ_d\\
        g = ƞ^2 + c^2 - 1
    \end{array}


In [ ]:
def cook_torrance(rho_d, theta_i, phi_i, theta_r, phi_r, ior, sigma):
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  n = np.array([0,0,1])
  h = halfway_vector(v_i, v_r)
  theta_h = np.arccos(np.dot(n,h))
  c = np.dot(v_r, h)
  g = np.square(ior) + np.square(c) - 1

  F = (np.square(g-c)/(2*np.square(g+c)))*(1+(np.square(c*(g+c)-1)/np.square(c*(g-c)+1)))
  G = min(
      1,
      (2*np.dot(n,h)*np.dot(n,v_r))/np.dot(v_r, h),
      (2*np.dot(n,h)*np.dot(n,v_i))/np.dot(v_r, h)
  )
  D = np.exp(-np.square((np.tan(theta_h)/sigma),2))/(4*np.square(sigma)*np.power(np.cos(theta_h),4))

  specular_term = (D*G*F)/(np.pi*np.cos(np.deg2rad(theta_r))*np.cos(np.deg2rad(theta_i)))
  diffuse_term = rho_d/np.pi

  return diffuse_term + specular_term

**GGX model**

It is actually a BSDF model, since it extends the microfacets theory below the surface.

**GGX model - distribution term**

It makes use of the Trowbridge-Reitz distribution:

\begin{align}
        D(\vec{h}) = \frac{α_g^2𝑋^+(\vec{h}⋅\vec{n})}{π(cosθ_h)^4(α_g^2+(tanθ_h)^2)^2}\\
    \end{align}
Where:
\begin{align}
        α_g \text{ is a width parameter}\\[1em]
        𝑋^+(x) = \left\{\begin{array}{cc}
        1 & if & x > 0\\
        0 & if & x ≤ 0
        \end{array}\right.
    \end{align}

**GGX model - attenuation term**

The shadowing and masking term has been derived from the microfacets distribution D:

\begin{align}
        G(\vec{v_i},\vec{v_r},\vec{h}, \vec{n}) ≈ G_1(\vec{v_i},\vec{h}, \vec{n})×G_1(\vec{v_r},\vec{h}, \vec{n})
    \end{align}

where:

\begin{align}
        G_1(\vec{v},\vec{h},\vec{n}) = 𝑋^+\left(\frac{\vec{v}⋅\vec{h}}{\vec{v}⋅\vec{n}}\right)\frac{2}{1+\sqrt{1+α_g^2+(tanθ_v)^2}}
    \end{align}

**GGX model - Fresnel term**

The Frenel term is the same one used in Cook-Torrance.

In [ ]:
def X_plus(x):
  if x > 0:
    return 1
  else:
    return 0

def G1(v, h, n, alpha_g):
  theta, phi = vector_angles(v[0], v[1], v[2])

  term1 = X_plus(np.dot(v,h)/np.dot(v,n))
  term2 = 2/(1+np.sqrt(1+np.square(alpha_g)+np.square(np.tan(np.deg2rad(theta)))))

  return term1 * term2

def ggx_model(rho_d, theta_i, phi_i, theta_r, phi_r, alpha_g, ior):
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  n = np.array([0,0,1])
  h = halfway_vector(v_i, v_r)
  theta_h = np.arccos(np.dot(n,h))
  c = np.dot(v_r, h)
  g = np.square(ior) + np.square(c) - 1

  F = (np.square(g-c)/(2*np.square(g+c)))*(1+(np.square(c*(g+c)-1)/np.square(c*(g-c)+1)))
  G = G1(v_i, h, n, alpha_g)*G1(v_r, h, n, alpha_g)
  D = (np.square(alpha_g)*X_plus(np.dot(h,n)))/(np.pi*np.power(np.cos(theta_h),4)*np.square(np.square(alpha_g)+np.square(np.tan(theta_h))))

  specular_term = (D*G*F)/(np.pi*np.cos(np.deg2rad(theta_r))*np.cos(np.deg2rad(theta_i)))
  diffuse_term = rho_d/np.pi

  return diffuse_term + specular_term

**Oren-Nayar model**

This model makes us of Lambertian diffuse microfacets to model rough, diffuse surfaces.

Given the surface roughness σ, the reflectance can be computed as follow:

\begin{align}
        f_r(\vec{v_i},\vec{v_r}) = \frac{\rho_d}{\pi}(A+B × max(0, cos(\Phi_i-\Phi_r)) × sin\alpha × tan\beta)
    \end{align}
Where:
\begin{array}{l}
        A = 1 - \left[\frac{\sigma^2/2}{\sigma^2+\frac{1}{3}}\right]\\
        B = \frac{\frac{9}{10}\sigma^2}{\sigma^2+\frac{9}{100}}\\
        \alpha = max(\theta_r, \theta_i)\\
        \beta = min(\theta_r, \theta_i)
    \end{array}

In [ ]:
def oren_nayar(rho_d, theta_i, phi_i, theta_r, phi_r, sigma):
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  n = np.array([0,0,1])
  h = halfway_vector(v_i, v_r)
  theta_h = np.arccos(np.dot(n,h))
  A = 1 - ((np.square(sigma)/2)/(1/3)+np.square(sigma))
  B = ((9/10)*np.square(sigma))/(np.square(sigma)+(9/100))
  alpha = max(theta_r, theta_i)
  beta = min(theta_r, theta_i)

  diffuse_term = rho_d/np.pi
  specular_term = (A + B*max(0, np.cos(np.deg2rad(phi_i - phi_r)))*np.sin(np.deg2rad(alpha))*np.tan(np.deg2rad(beta)))
  return diffuse_term * specular_term

In [ ]:
theta_r = np.degrees(np.pi/4)
phi_r = np.degrees(3*np.pi/2)

theta_i = np.degrees(np.pi/4)
phi_i = np.degrees(np.pi/2)

rho_d = 0.6
sigma = np.pi/6

vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
v_i = np.array([vi_x, vi_y, vi_z])
v_r = np.array([vr_x, vr_y, vr_z])

print(f"Halfway Vector = {halfway_vector(v_i, v_r)}")
print(f"BRDF = {oren_nayar(rho_d, theta_i, phi_i, theta_r, phi_r, sigma)}")

Halfway Vector = [-6.123234e-17  0.000000e+00  1.000000e+00]
BRDF = 0.060086237810699696


In [ ]:
# vector set 2
v_r = np.hstack((-0.936, -0.341, 0.087))
v_i = np.hstack((-0.168, -0.951, 0.259))

rho_d = 0.9
sigma = 2*np.pi/9
theta_i, phi_i = vector_angles(-0.168, -0.951, 0.259)
theta_r, phi_r = vector_angles(-0.936, -0.341, 0.087)
print(f"Halfway Vector = {halfway_vector(v_i, v_r)}")
print(f"BRDF = {oren_nayar(rho_d, theta_i, phi_i, theta_r, phi_r, sigma)}")

Halfway Vector = [-0.63656812 -0.74496921  0.19950414]
BRDF = 0.3421325824009256


## Rendering Equation and Ray Casting

**Phong model - revised**

Phong - Original:

\begin{array}{l}
        f_r(\vec{v_i}, \vec{v_r}) = k_s(\vec{v_r}·\vec{r_{v_i}})^m\\
        f_r(\vec{v_i}, \vec{v_r}) = \frac{k_d}{\pi}+k_s(\vec{v_r}·\vec{r_{v_i}})^m\\
    \end{array}

Modified Phong:
\begin{array}{l}
        f_r(\vec{v_i}, \vec{v_r}) = \frac{k_d}{\pi}+k_s\frac{m+2}{2\pi}(\vec{v_r}·\vec{r_{v_i}})^m\\
    \end{array}

In [ ]:
def modified_phong(k_s, k_d, theta_i, phi_i, theta_r, phi_r, m):
  vi_x, vi_y, vi_z = vector_coordinates(theta_i, phi_i)
  vr_x, vr_y, vr_z = vector_coordinates(theta_r, phi_r)
  v_i = np.array([vi_x, vi_y, vi_z])
  v_r = np.array([vr_x, vr_y, vr_z])

  n = np.array([0,0,1])
  # calculate mirror vector
  r_vi = v_i - 2 * np.dot(v_i, n) * n

  diffuse_term = k_d/np.pi
  specular_term = k_s * ((m+2)/(2*np.pi)) * np.power(np.dot(v_r, r_vi), m)

  return diffuse_term + specular_term

**The reflection equation**

This equation describes the relationship between the incident and reflected radiance. It is known as local illumination model.

\begin{array}{l}
        f_r(p,\vec{v_i}, \vec{v_r}) = \frac{dL_r(p,\vec{v_r})}{L_i(p,\vec{v_i})(\vec{n}⋅\vec{v_i})dω_i} \\→ dL_r(p,\vec{v_r}) = f_r(p,\vec{v_i}, \vec{v_r})L_i(p,\vec{v_i}(\vec{n}⋅\vec{v_i})dω_i)\\
        \text{where:}\\
        Lr(p,\vec{v_r}) = ∫_{Ω+}L_i(p,\vec{v_i}) f_r(p,\vec{v_i}, \vec{v_r})(\vec{n}⋅\vec{v_i})dω_i)\\
        L_o(p,\vec{v_r}) = L_e(p,\vec{v_r})+L_r(p,\vec{v_r})
    \end{array}

  - L<sub>o</sub> = total outgoing radiance towards the viewer
  - L<sub>e</sub> = emitted radiance
  - L<sub>r</sub> = reflected radiance
  - p = point of reflection/emission


  **Refraction**

 Snell's law: ƞ<sub>1</sub>sinθ<sub>1</sub> = ƞ<sub>2</sub>sinθ<sub>2</sub>

 **Rays**

 In rendering, a ray is specified as an interval on a line:

 \begin{array}{l}
        point(t) = origin + t * direction\\
        B = A+t*direction
    \end{array}
  
Ray casting function *r* is given a point *p* and a direction *v*, returns *p'*, the point on the closest surface from p in the direction of *v*:

 \begin{array}{l}
        r(p,\vec{v}) = p'
    \end{array}



### Light sources


**Equirectangular map**

Mapping coordinates from unit direction in the world (e.g. Sphere) to Equirectangular image(Latitud-longitude):

1. (u,v) coordinates in the image, with u ∈ [0,2], v ∈ [0,1]
2. D = (D<sub>x</sub>,D<sub>y</sub>,D<sub>z</sub>) unit direction in the world (sphere)
3.Coordinate system:
  - [0,0,1] = right-handed
  - [1,0,0] = forward
  - [0,1,0] = up
4. Mapping equation:

\begin{align}
        (u,v) = (1+\frac{1}{π}atan2(D_x,-D_z),\frac{1}{π}cos^{-1}D_y)
    \end{align}

In [ ]:
def equirectangular_map(D_x, D_y, D_z):
  u = 1 + (1/np.pi)*(np.arctan2(D_x, -D_z))
  v = (1/np.pi)*(np.arccos(D_y))
  return u,v

### Rays and Reflection

**Rays**

- A ray has an origin p<sub>0</sub> and direction d, where |d| = 1:
- Any point along the path of the ray p<sub>t</sub> can be calculated by:

\begin{array}{l}
        p_t = p_0 + t\vec{d}\\
        p_{t,x} = p_{0,x} + td_x\\
        p_{t,y} = p_{0,y} + td_y\\
        p_{t,z} = p_{0,z} + td_z
    \end{array}



In [ ]:
def ray(t, p_0, d):
  p_x, p_y, p_z = p_0[0], p_0[1], p_0[2]
  d_x, d_y, d_z = d[0], d[1], d[2]

  x = p_x + t*d_x
  y = p_y + t*d_y
  z = p_z + t*d_z

  return x,y,z

**Specular Reflection**

Given a ray with direction d, the reflected direction will be r<sub>d</sub> at the normal to the surface.

Where
- ||d|| = 1 and ||n|| = 1:
- d' = -d
- r<sub>d</sub> is the mirror direction of d'
- θ is the angle between d' and n: (d'⋅n)

\begin{align}
        r_d = d - 2(n⋅d)n
    \end{align}

In [ ]:
def specular_reflection(d, n):
  return d - 2*np.dot(n,d)*n

### Ray-Sphere intersection



A point S on a sphere given in parametric form is:

\begin{align}
        S(θ,Φ) = r(sinθcosΦ, sinθsinΦ, cosθ)
    \end{align}

All points on a sphere are at the same distance from the centre (x<sub>c</sub>,y<sub>c</sub>,z<sub>c</sub>)

A point on a sphere given in implicit form:

\begin{array}{l}
        r = \sqrt{(x-x_c)^2 + (y-y_c)^2 + (z-z_c)^2}\\
        (x-x_c)^2 + (y-y_c)^2 + (z-z_c)^2 - r^2 = 0
    \end{array}

**Intersecting implicit with parametric**

1. Ray:
\begin{align}
          p_t = p_0 + t\vec{d} \left\{\begin{array}{l}
          p_{t,x} = p_{0,x} + td_x\\
          p_{t,y} = p_{0,y} + td_y\\
          p_{t,z} = p_{0,z} + td_z
          \end{array}\right.
    \end{align}

2. Substitute into point on the sphere's implicit form:
\begin{array}{l}
        (p_{0,x} + td_x-x_c)^2 + (p_{0,y} + td_y-y_c)^2 + ( p_{0,z} + td_z-z_c)^2 - r^2 = 0
    \end{array}

3. Rewrite form as a quadratic at<sup>2</sup> + bt + c = 0:
\begin{array}{l}
        a = d_x^2 + d_y^2 + d_z^2\\
        b = 2(d_x(p_{0,x} - x_c) + d_y(p_{0,y} - y_c) + d_z(p_{0,z} - z_c))\\
        c = (p_{0,x} - x_c)^2 + (p_{0,y} - y_c)^2 + (p_{0,z} - z_c)^2 - r^2
    \end{array}

4. Check how many intersections between the ray and sphere:
\begin{array}{l}
        at^2 + bt +c = 0 → t = \frac{-b±\sqrt{b^2-4ac}}{2}\\
        \text{where:}\\
        b^2-4ac < 0 = \text{none}\\
        b^2-4ac = 0 = \text{one}\\
        b^2-4ac > 0 = \text{two}
    \end{array}


In [ ]:
def ray_sphere_intersection(p_0, d, x_c, y_c, z_c, r):
  p_x, p_y, p_z = p_0[0], p_0[1], p_0[2]
  d_x, d_y, d_z = d[0], d[1], d[2]

  a = np.sum(np.square(d))
  b = 2*((d_x*(p_x-x_c)) + d_y*(p_y-y_c) + d_z*(p_z-z_c))
  c = np.square(p_x-x_c) + np.square(p_y-y_c) + np.square(p_z-z_c) - np.square(r)

  sqrt_term = np.square(b) - 4*a*c
  print(f"a = {a}, b = {b}, c = {c}, sqrt(b^2 - 4ac) = {sqrt_term}")

  if sqrt_term < 0:
    print("No intersect")
    return None
  if sqrt_term == 0:
    t = -b/2
    print(f"One intersect: t={t}")
    return ray(t, p_0, d)
  if sqrt_term > 0:
    t_0 = (-b-np.sqrt(sqrt_term))/2
    t_1 = (-b+np.sqrt(sqrt_term))/2
    print(f"Two intersects: t_0={t_0}, t_1={t_1}")
    return ray(t_0, p_0, d), ray(t_1, p_0, d)

### Ray-Plane and Ray-Triangle intersection



**Equation of a Plane**

\begin{array}{l}
      ax + by + cz + d = 0\\
      n = \begin{pmatrix}a,b,c\end{pmatrix}\\
      p = (x_0, y_0, z_0)\\
      n⋅p + d = 0\\
      d = -(ax_0 + by_0 + xz_0)\\
      ax + bx + cz - (ax_0 + by_0 + xz_0) = 0\\
      a(x-x_0) + b(y-y_0) + c(z-z_0) = 0
    \end{array}

Where:
- n is a normal vector to the plane
- p is a point on the plane

If n is unknown, 3 points are required to find the equation of the plane. The points must not be in a line:

\begin{array}{l}
        \vec{p_1} = [x_1, y_1, z_1]\\
        \vec{p_2} = [x_2, y_2, z_2]\\
        \vec{p_3} = [x_3, y_3, z_3]\\
        \vec{u} = \vec{p_1} - \vec{p_2} = [u_x, u_y, u_z]\\
        \vec{v} = \vec{p_3} - \vec{p_2} = [v_x, v_y, v_z]\\
        n = \vec{v} × \vec{u}
    \end{array}

In [ ]:
def plane_with_normal(n, p_0):
  a,b,c = n[0], n[1], n[2]
  x_0, y_0, z_0 = p_0[0], p_0[1], p_0[2]

  x = a
  y = b
  z = c
  d = -np.dot(n, p_0)

  print(f"Equation of Plane: {x}x + {y}y + {z}z + {d} = 0")
  return x, y, z, d

def plane_without_normal(p1, p2, p3):
  u = p1-p2
  v = p3-p2
  n = np.cross(u, v)
  n = n/np.linalg.norm(n)
  return plane_with_normal(n, p1)

In [ ]:
# Test with Examples
n = np.array([0.539, 0.770, 0.342])
p_0 = np.array([3, -2, 4])

plane_with_normal(n, p_0)

p1 = np.array([2, 1.5, 1])
p2 = np.array([2, 1.5, 3.5])
p3 = np.array([0.5, 1.5, 3.5])

plane_without_normal(p1, p2, p3)

Equation of Plane: 0.539x + 0.77y + 0.342z + -1.445 = 0
Equation of Plane: 0.0x + 1.0y + 0.0z + -1.5 = 0


(0.0, 1.0, 0.0, -1.5)

**Ray-Plane intersection**

Ray:
\begin{align}
        p_t = p_0 + td_{ray}
    \end{align}

Point on a plane:
\begin{array}{l}
        n\cdot p_t + d = 0\\
        n⋅(p_0 + td_{ray}) + d = 0\\
        n⋅p_0 + n⋅td_{ray} + d = 0
    \end{array}
Solve for t:
\begin{align}
        t = -\frac{(n⋅p_0 + d)}{n⋅d_{ray}}
    \end{align}

In [ ]:
def ray_plane_intersection(p_0, d_ray, n, d):
  t = -((np.dot(n, p_0)+d)/np.dot(n, d_ray))
  return ray(t, p_0, d_ray)

**Ray-Triangle intersection**

Given a triangle with points p<sub>1</sub>, p<sub>2</sub>, p<sub>3</sub>, check p<sub>t</sub> if is inside the triangle:

\begin{array}{l}
        (\vec{p_2 - p_1} × \vec{p_t - p_1}) ⋅n ≥ 0\\
        (\vec{p_3 - p_2} × \vec{p_t - p_2}) ⋅n ≥ 0\\
        (\vec{p_1 - p_3} × \vec{p_t - p_3}) ⋅n ≥ 0\\
    \end{array}

If all conditions are met, p<sub>t</sub> intersects with the triangle.

In [ ]:
def ray_triangle_intersection(p1, p2, p3, d_ray, n, d,p_0):
  p_x,p_y,p_z = ray_plane_intersection(p_0, d_ray, n, d)
  p_t = np.array([p_x,p_y,p_z])

  if (
      (np.dot(np.cross((p2-p1, p_t-p1)),n) >= 0) &
      (np.dot(np.cross((p3-p2, p_t-p2)),n) >= 0) &
      (np.dot(np.cross((p1-p3, p_t-p3)),n) >= 0)
  ):
    return p_t
  else:
    return None

In [ ]:
d_ray = np.array([-0.996, 0.0857, 0])
n = np.array([0.966, 0.259, 0])
p_m = np.array([-14, -5.995, 0])

# A ray with direction d_ray hits a mirror with normal n.
print(f"A ray with direction: {d_ray} hit a mirror at point {p_m} with the normal {n}.")
d_ray2 = specular_reflection(d_ray, n)
print(f"The ray is reflected in the direction {d_ray2} from point {p_m}")
n_slab = np.array([-1, 0 ,0])
v4 = np.array([-10, 2, -2])
x, y, z, d = plane_with_normal(n_slab, v4)
p_3 = ray_plane_intersection(p_m, d_ray2, n_slab, d)
theta_i = np.arccos(np.dot(d_ray2, -n_slab))
print(f"The reflected ray hits a slab on the above plane. The ray hits the slab at {p_3} with an angle of {theta_i}rads")
ior_slab = 1.81
ior_air = 1.0
theta_r = np.arcsin((ior_air/ior_slab) * np.sin(theta_i))
d_ray3 = np.array([np.cos(theta_r), np.sin(theta_r), 0])
print(f"Using snell's law, the ray is refrected in the slab travelling at {theta_r} degrees, the new direction is {d_ray3}")
p_4 = p_3 + np.array([4, 4*np.tan(theta_r), 0])
print(f"The ray then should leave the slab at point {p_4} and return to it's previous direction of {d_ray2}")
s_c = np.array([10, 5, -3])
s_r = 5
print("Ray-Sphere intersection:")
ray_sphere_intersection(p_4, d_ray2, s_c[0], s_c[1], s_c[2], s_r)

A ray with direction: [-0.996   0.0857  0.    ] hit a mirror at point [-14.     -5.995   0.   ] with the normal [0.966 0.259 0.   ].
The ray is reflected in the direction [0.8199635  0.57258876 0.        ] from point [-14.     -5.995   0.   ]
Equation of Plane: -1x + 0y + 0z + -10 = 0
The reflected ray hits a slab on the above plane. The ray hits the slab at (-10.0, -3.2017597432291756, 0.0) with an angle of 0.609449075080123rads
Using snell's law, the ray is refrected in the slab travelling at 0.3217759180535049 degrees, the new direction is [0.94867528 0.31625183 0.        ]
The ray then should leave the slab at point [-6.         -1.86831368  0.        ] and return to it's previous direction of [0.8199635  0.57258876 0.        ]
Ray-Sphere intersection:
a = 1.0001980353343753, b = -34.104270504441104, c = 287.17373282856414, sqrt(b^2 - 4ac) = 14.178853141016816
Two intersects: t_0=15.169394355215815, t_1=18.93487614922529


((6.438349694450762, 6.817511092004322, 0.0),
 (9.525907326959242, 8.97358366056, 0.0))

In [ ]:
n_slab = np.array([-1, 0 ,0])
v4 = np.array([-10, 2, -2])
x, y, z, d = plane_with_normal(n_slab, v4)
d_ray2 = np.array([0.820, -0.572, 0])
p_0 = np.array([-12, -4.6, 0])
p_3 = ray_plane_intersection(p_0, d_ray2, n_slab, d)
print(f"The reflected ray hits a slab on the above plane. The ray hits the slab at {p_3} with an angle of {theta_i}rads")
ior_slab = 1.31
ior_air = 1.0
theta_r = np.arcsin((ior_air/ior_slab) * np.sin(theta_i))
d_ray3 = np.array([np.cos(theta_r), np.sin(theta_r), 0])
print(f"Using snell's law, the ray is refrected in the slab travelling at {theta_r} degrees, the new direction is {d_ray3}")
p_4 = p_3 + np.array([4, 4*np.tan(theta_r), 0])
print(f"The ray then should leave the slab at point {p_4} and return to it's previous direction of {d_ray2}")
s_c = np.array([10, 5, -3])
s_r = 5
print("Ray-Sphere intersection:")
ray_sphere_intersection(p_4, d_ray2, s_c[0], s_c[1], s_c[2], s_r)

Equation of Plane: -1x + 0y + 0z + -10 = 0
The reflected ray hits a slab on the above plane. The ray hits the slab at (-10.0, -5.995121951219511, 0.0) with an angle of 0.609449075080123rads
Using snell's law, the ray is refrected in the slab travelling at 0.45221464065639344 degrees, the new direction is [0.8994816  0.43695863 0.        ]
The ray then should leave the slab at point [-6.         -4.05196433  0.        ] and return to it's previous direction of [ 0.82  -0.572  0.   ]
Ray-Sphere intersection:
a = 0.9995839999999998, b = -15.884552805252149, c = 321.9380582510232, sqrt(b^2 - 4ac) = -1034.8975102523189
No intersect


## Ray Tracing and Path Tracing

**Ray casting:** The process of finding the closest object along a ray. If a ray hits a point, the pixel colour is estimated using shading models.

**Phong shading:** The final colours is composed of three components:

- Ambient colour
- Diffuse colour
- Specular colour

\begin{align}
          L_r = k_aI_a + \left[
          \frac{k_d}{\pi} (\vec{n} ⋅ \vec{v_i}) + k_s \frac{m+2}{2\pi} (\vec{v_r} ⋅ \vec{r_{vi}})^m
          \right] \frac{L_i}{r^2}
    \end{align}

**Shading with sRGB Lambertian Reflectance**

where:
- K<sub>a</sub>, K<sub>d</sub>, K<sub>s</sub> : ambient, diffuse and specular coefficients.
- I<sub>a</sub> : ambient colour
- L<sub>i</sub> : incoming radiance colour
- ρ<sub>d</sub> : diffuse albedo colour
- θ<sub>i</sub> : incoming radiance angle

Compute the Lambertian reflectance for each RGB component channel, ch.

\begin{array}{l}
          L_{r,ch} = k_aI_{a,ch} + \frac{ρ_{d,ch}}{\pi} (\vec{n} ⋅ \vec{v_i}) \frac{L_{i,ch}}{r^2}\\
          (\vec{n} ⋅ \vec{v_i}) = sinθ_i
    \end{array}



In [ ]:
def RGB_lambertian_reflectance(i_a, k_a, L_i, rho_d, theta_i, r):
  L_r = np.empty((3))
  for ch in range(3):
    ambient_component = i_a[ch]*k_a
    diffuse_component = (rho_d[ch]/np.pi)*np.sin(np.deg2rad(theta_i))*(L_i[ch]/np.square(r))
    L_r[ch] = ambient_component + diffuse_component
  return L_r

In [ ]:
# Shading with sRGB example with Lambertian Reflectance
i_a = np.array([0.1, 0.2, 0.3])
k_a = 0.15
L_i = np.array([0.4, 0.15, 0.1])
rho_d = np.array([0.1, 0.35, 0.1])
theta_i = 30
r = 1

RGB_lambertian_reflectance(i_a, k_a, L_i, rho_d, theta_i, r)

array([0.0213662 , 0.03835563, 0.04659155])

### Monte Carlo Integration

### Sampling the Hemisphere

**Uniform directional sampling - rejection method**

To generate a sample:
1. Generate 3 random numbers, (x,y,z) in [-1, 1]
2. Compute s = x<sup>2</sup> + y<sup>2</sup> + z<sup>2</sup>, reject if s > 1
3. Normalise (x,y,z) to get a ray r.
4. If r⋅n < 0: flip the direction of r.

**Uniform directional sampling - inversion method**

To generate a sample:
1. Generate Φ in [0, 2π]
2. Generate z in [0, 1]
3. Set θ = cos<sup>-1</sup>z
4. r = coordinates (θ, Φ)
5. Express r in cartesian coordinates as: r = (x,y,z) = (sinθcosΦ, sinθsinΦ, cosθ)
6. r us aligned with the local surface normal, rotate r using the rotation matrix R:

\begin{array}{l}
        R = \begin{pmatrix}
        cosαcosβ & -sinβ & sinαcosβ\\
        sinβcosα & cosβ & sinαsinβ\\
        -sinα & 0 & cosα
        \end{pmatrix}\\
        r' = r × R
    \end{array}

Where:
- α is the angle between the sphere tangent and xy component of r
- β is the angle between the normal and r

\begin{array}{l}
        α = cos^{-1}r_z\\
        β = tan^{-2}(r_x,r_y)
    \end{array}

### Specular BRDFs

**Importance sampling for Phong model**

Phong model - energy conservative:

\begin{align}
        f_r(\vec{v_i},\vec{v_r}) = \frac{k_d}{\pi} + k_s \frac{m+2}{2π}(\vec{v_i}⋅\vec{r_{vi}})^m
    \end{align}

To generate a sample:
1. set (v<sub>i</sub>,v<sub>r</sub>) to cosδ and sample for δ
2. δ = cos<sup>-1</sup>(Z<sup>1/m+1</sup>) to account for the exponent m in the BRDF.
3. Generate Φ<sub>δ</sub> in [0,2π]
4. Convert to cartesian coordinates(x,y,z)
5. This time the direction must be aligned with r<sub>vi</sub>

# Topic 4

## Geometric Vision

### Point Clouds

**3D Representation: Point Clouds**

A point cloud is an unstructured set of 3-dimensional coordinates:

\begin{array}{l}
        P = \{v_1,...,v_n\}, \text{where } v_i ∈ \mathbb{R}^3
    \end{array}

**3D Representation: Depth maps**

A depth map respresents a 3D scene from the perspective of a camera.

For each pixel (x,y), the the w value of the (u,v,w) point where a ray through that pixel intersects the scene.

A depth map can be converted to a point cloud by projecting along each ray to provide a 3D world point:

\begin{align}
        W(x,y) = \begin{pmatrix}
        \frac{x-c_x}{f}w(x,y) \\
        \frac{y-c_y}{f}w(x,y) \\
        w(x,y)
        \end{pmatrix}
    \end{align}

### Structure from Motion

View a fixed scene or object *I* from multiple positions *C<sub>i</sub>* of a camera. Use perspective viewing geometry to estimate a 3D model.

Unknowns:
- 3D position, w<sub>i</sub> ∈ ℝ<sup>3</sup> of *I* world points
- Pose, R, t, of each camera C

Knowns:
- Intrinsic camera matrix K
- Position X in each camera images that corresponds to the same world point.

Typical Algorithm:

1. Choose an initial pair of images.
2. Detect and match features, fit fundamental matrix to matches with RANSAC, decompose fundamental matrix to get relative pose between cameras.
3. Triangulate matched features to get initial 3D world poisitions of features.
4. While there are still new views to add:
  - Extract features from new image and match to previous views.
  - Solve perspective projection to compute pose of new camera view to existing world model.
  - Triangulate position of new features marched for the first time.
  - Bundle adjust over all camera poses and 3D world points.

### Binocular Stereo

Objective: reconstruct a dense depth map from a pair of overlapping images. For every pixel in the left image, find a match in the right image allowing triangulation of every pixel.

Pipeline:
1. Calibrate by computing "relative" pose between each camera C to convert image coordinates from C<sub>1</sub> to C<sub>2</sub>.
2. Warp images using homographies such that the match for a pixel will lie on the same row in the other image.
3. For each pixel in the left image, search along row of pixel in the right image to find the best match.
4. The desparity of x'<sub>c1</sub> - x'<sub>c2</sub> is inversely proportional to the depth w.
  - x' is the warped x component from pixel X of the image I.
  - x'<sub>c1</sub>, x'<sub>c2</sub> are the matching pixels from the warped image along the same y-axis.

**Multiview Stereo**

Assume structure-from-motion provides estimates for intrinsic and extrinsic camera parameters.

**Shape-from-silhouette**

1. Compute binary foreground images for each viewpoint. (Mask the foreground)
2. Divide object space into 3D grid of voxels.
3. Each voxel is intesected with each silhouette volume.
4. Only voxels that lie inside all silouette volumes remain part of the final shape.

The shape that remains is known as the visual hull.

**Space carving**

If a voxel is visible in two or more images, it should appear the same in all images.

**Pollefeys and Van Gool algorithm**

1. Apply structure-from-motuon to image collection
2. For each pair of overlapping images, apply binocular stereo to get dense depth map.
3. Project each depth map to a point cloud
4. Merge point clouds
5. Reconstruct smooth surface from point clouds.

## Photometric Methods

**Interaction between light and surface**

- Light of radiance L<sub>i</sub> comes from a light source at an incoming direction θ<sub>i</sub>
- It sends out a ray of radiance L<sub>r</sub> in the outgoing direction θ<sub>r</sub>

\begin{align}
        L_r = f_r(\vec{v_i},\vec{v_r})L_i cosθ_i
    \end{align}
- L<sub>r</sub> = Output radiance along θ<sub>r</sub>
- L<sub>i</sub> = Incoming irradiance along θ<sub>i</sub>
- f<sub>r</sub>(v<sub>i</sub>,v<sub>r</sub>) = BRDF

Special case 1: Perfect mirror
\begin{align}
        f_r(\vec{v_i},\vec{v_r})= 0 \text{ unless } θ_i = \theta_r
    \end{align}
- Also called "specular surfaces"
- Reflects light in a single, particular direction.

Special case 2: Matte surface:
\begin{align}
        f_r(\vec{v_i},\vec{v_r})= ρ_0
    \end{align}
- Also called "Lambertian surfaces"
- Reflected light is independent of viewing direction

**Lambertian Surfaces**
\begin{align}
        L_r = ρL_i cosθ_i = ρL_i(v_i⋅v_r)
    \end{align}
- ρ is the albedo, represents colour/brightness.

Intrinsic Image Decomposition:
\begin{align}
        I(x,y) = ρ_{(x,y)}L_i(v_i⋅v_r)_{(x,y)}
    \end{align}
Where:
- ρ<sub>(x,y)</sub> = Reflectance image
- Li(v<sub>i</sub>⋅v<sub>r</sub>)<sub>(x,y)</sub> = Shading image

**Photometric Stereo**

Recovery from multiple images:

\begin{align}
        I(x,y) = ρ_{(x,y)}L_i(v_i⋅v_r)_{(x,y)}
    \end{align}

- Represents an equation in the albedo and normals
- Multiple images give constraints on albedo and normals

Photometric stereo is a technique for estimating surface orientation of objects by observing that object under different lighting conditions.

**The Math**

\begin{array}{c}
        I_1 = L_1^TG\\
        I_2 = L_2^TG\\
        ⋮\\
        I_n = L_n^TG
    \end{array}
- I = vector of intensities
- L = vector of lighting directions
- G = vector of unknowns

Solvable by least squares.

### Photometric Stereo

**Photometric stereo and BRDFs**

Considering only reflected light, radiance towards a viewer:

\begin{align}
        L_r(v_r) = ∫_{Ω_+}f_r(v_i,v_r)L_i(v_i)cosθ_idv_i
    \end{align}
This describes an arbitarily complicated illumination where L<sub>i</sub>(v<sub>i</sub>) varies for every possible direction.

Simpler with a single point source with radiance L<sub>i</sub> coming from direction v<sub>i</sub>:

\begin{align}
        L_r(v_r) = f_r(v_i,v_r)L_icosθ_i
    \end{align}

Simplest possible BRDF is perfect diffuse (Lambertian) reflector:

\begin{align}
        L_r(v_r) = \frac{ρ_d}{\pi}
    \end{align}

We can roll all of the constants into a scale factor on s:

\begin{align}
        s = \frac{L_i}{\pi}v_i
    \end{align}

- ρ<sub>d</sub> ∈ [0,1] is the diffuse albedo
  - 0 = perfect black
  - 1 = perfect white
  - Usually RGB dependent per channel = "colour" of an object.

**Lambertian Photometric Stereo**

Objective: recover the surface normal n and diffuse albedo ρ<sub>d</sub> for each pixel

1. Observe the same object n ≥ 3 times, each with a point light source from directions s<sub>i</sub>
2. For each pixel, observe image intensity I<sub>i</sub>
3. Each image intensity give the linear equation:
\begin{align}
        I_i = s_i^⊤x
    \end{align}
    - unknown vector x = ρ<sub>d</sub>n
4. Combine all observations for one pixel into a linear system of equations:
\begin{align}
        \begin{pmatrix}I_1\\⋮\\I_n\end{pmatrix}=
        \begin{pmatrix}s_1^⊤\\⋮\\s_n^T\end{pmatrix}x
    \end{align}
5. Solve with linear least squares.
6. ρ<sub>d</sub> = ||x||
7. n = x/||x||

In [ ]:
def lambertian_photometric_stereo(S, images, mask):
  '''
  S = light source directions for each image
  images = The images to compute
  '''
  nimgs = len(images)
  img_h = images.shape[1]
  img_w = images.shape[2]
  Sscaled = np.zeros((20, 3))
  albedo = np.zeros((img_h, img_w))
  normal = np.zeros((img_h, img_w, 3))

  for i in range(nimgs):
    S[i,:] = S[i,:] / np.linalg.norm(S[i,:])

  for i in range(nimgs):
    intensities = images[i,:,:]
    intensities = intensities[mask]
    maxval = np.percentile(intensities, 99)
    Sscaled[i,:] = S[i,:]*maxval

  for row in range(img_h):
    for col in range(img_w):
      if mask[row, col]:
        I = images[:, row, col]
        x = S_plus @ I
        albedo[row, col] = np.linalg.norm(x)
        normal[row, col, :] = x / albedo[row,col]

  return albedo, normal


### Analysis-by-synthesis



Recover per scene from n ≥ 1 images:

\begin{array}{cl}
        min(Θ) \sum^n_{i=1} ||render(Θ_i) - I_i|| & Θ=\{Θ_1,...,Θ_n\}
    \end{array}

Where Θ contains:
- Geometry
- Material properties (parameters of a BRDF)
- Camera intrinsic and extrinsic parameters
- Illumination

There can be an infinite combination of solutions to describe a scene. Need to constrain the problem.

- Each variable to be recovered is an optimisation variable.
- Iteratively adjust to reduce a loss.
- Non-linear optimisation problem: Need to be able to compute derivatives.


### Differentiable rendering

Use soft rasterisation to transform discrete problems to differentiable problems that can calculate a loss.

### Deep learning for Inverse Rendering

- A deep neural network is just a function, f.
- The behaviour of the function depends on some parameters Θ. The function is written as f<sub>Θ</sub>



# Topic 5

## Participating Media

### Light scattering participating media

- A participating medium can be modelled as a collection of microscopic particles, randomly located within the medium.
- In lighting simulations, we consider the probabilistic behaviour as light travels through the medium, rather than representing each individual particle.
- The probability of interaction between light and particles can be described using the extinction coefficient σ<sub>t</sub> of the medium, measured in 1/m.

**Absorption and out-scattering**

Two kinds of interactions decrease the radiance along a ray:
1. The photon may be absorbed by the particle:
\begin{align}
        (\vec{v}·∇_a)L(x,\vec{v}) = -σ_a(x)L(x,\vec{v})
    \end{align}
  - σ<sub>a</sub> : absorption coefficient = probability of absorption
  
2. The photon may be scattered in another direction:
\begin{align}
        (\vec{v}·∇_s)L(x,\vec{v}) = -σ_s(x)L(x,\vec{v})
    \end{align}
  - σ<sub>s</sub> : absorption coefficient = probability of scattering

Both interactions lead to a change in radiance along a ray.

σ<sub>a</sub> + σ<sub>s</sub> = σ<sub>t</sub>

**Emission and in-scattering**

Two kinds of interactions increase the radiance along a ray:
1. A medium can emit radiance:
\begin{align}
        (\vec{v}·∇_e)L(x,\vec{v}) = σ_a(x)L_e(x,\vec{v})
    \end{align}

2. Increase in radiance due to in-scattering of radiance from other directions:
\begin{align}
        (\vec{v}·∇_s)L(x,\vec{v}) = σ_s(x)L_{in}(x,\vec{v})
    \end{align}

The ratio σ<sub>s</sub>/σ<sub>t</sub> = scattering albedo. Represents probability of a photon scattering at a particular location, x, in the medium.

**Radiative transfer**

To quantify the total change in radiance along the ray at a given position x within the medium, solve the radiative transfer equation:

\begin{align}
        (\vec{v}·∇)L(x,\vec{v}) = -σ_a(x)L(x,\vec{v}) + -σ_s(x)L(x,\vec{v}) + σ_a(x)L_e(x,\vec{v}) + σ_s(x)L_{in}(x,\vec{v})
    \end{align}

**The phase function**

The role of the phase function is similar to the BRDF in the context of participating media. It describes where light travels after hitting a particle within the medium.

\begin{align}
        p(x, \vec{v_i}, \vec{v_s})
    \end{align}

**Henyey-Greenstein (HG) Phase Function**

\begin{align}
        p_{HG}(x,\theta) = \frac{1-g^2}{4\pi(1+g^2-2gcos\theta)^{1.5}}
    \end{align}
- g = 0 corresponds to isotropic scattering (equal in all directions)
- 0 < g ≤ 1 corresponds to forward scattering
- -1 ≤ g < 0 corresponds to backward scattering



### Lambertian surface reflection

**Surface normal from spherical gradients**

Images under 3 different spherical lighting directions, x, y, z.

Lambertian reflectance:

\begin{align}
      R(\vec{ω}, \vec{n}) = ρF(\vec{ω}, \vec{n})
\end{align}

The reflectance L<sub>i</sub> from a view direction v under spherical illumincation P<sub>i</sub> is:

\begin{array}{l}
      L_i(\vec{v}) = \int_Ω P_i(\vec{ω})R(\vec{ω}, \vec{n})d\vec{ω})\\
      P_x(\vec{ω}) = ω_x
\end{array}

R does not depend on the first two components of ω, hence:

\begin{array}{l}
        L_x(\vec{v}) = n_x\left( \frac{2\pi\rho}{3} \right)\\
        L_y(\vec{v}) = n_y\left( \frac{2\pi\rho}{3} \right)\\
        L_z(\vec{v}) = n_z\left( \frac{2\pi\rho}{3} \right)\\
    \end{array}